<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/tnj_with_null_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
from tqdm.notebook import tqdm


def Search_Some(dataset,some):
  dic={}
  for col in tqdm(list(dataset.columns)):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == some:
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic

def fun_split(X,Y,test_size,feature):

    f=Y.value_counts()
    rows=X.shape[0]
    initial_list=[]
    count=0
    for i in f:
        initial_list.append(i)
    train_size=1-test_size
    start=0

    switch=False
    flage = False

    for i in tqdm(initial_list):

        train=start+int(i*train_size)
        f= X[feature][train]
        count1=0
        count2=0
        index1=train
        index2=train+1
        end=start+i-1
        
        while X[feature][index1]==f:
            if index1<start:
                break
            index1-=1
            count1+=1
            
            
        while X[feature][index2]==f:
            if index2>end:
                break
            index2+=1
            count2+=1

        if count1>=count2:
            train+=count2
        else:
            train-=count1
        
        
        if flage == False:
            X_train= X[start:train+1]
            X_test= X[train+1:end]
            Y_train = Y[start:train+1]
            Y_test = Y[train+1:end]
            flage=True

        else:
            X_train=pd.concat([X_train, X[start:train+1]], ignore_index=True)
            X_test=pd.concat([X_test, X[train+1:end+1]], ignore_index=True)
            Y_train=pd.concat([Y_train, Y[start:train+1]], ignore_index=True)
            Y_test=pd.concat([Y_test, Y[train+1:end+1]], ignore_index=True)

        start=start+i

    print('Train Percentage:',X_train.shape[0]/rows)

    return X_train,X_test,Y_train,Y_test

In [27]:
import pandas as pd
import numpy as np

In [28]:
data= pd.read_excel('/content/drive/MyDrive/Tanjila_mam/DS-Healthcare.xlsx')
target = 'Type'

In [29]:
data.columns = data.iloc[0]               #columns have been set as index 0
data = data[1:]  #new dataset index starts from index 1
data = data.reset_index(drop=True)

In [ ]:
data.head()

,Subject,Type,Cycle,Time,RR-I,HR,R-H,P-H,QRS,PRQ,QT,QTC,ST
0,E1,ischemic,1,1.072,0.932,64.377682,3.305,3.81,0.088,0.16,0.384,0.397762,0.296
1,E1,ischemic,2,2.004,1.004,59.760956,3.285,3.81,0.1,0.16,0.64,0.638724,0.54
2,E1,ischemic,3,3.008,0.996,60.240964,3.34,3.83,0.128,0.148,0.628,0.62926,0.5
3,E1,ischemic,4,4.004,1.016,59.055118,3.325,3.81,0.108,0.172,0.388,0.384933,0.28
4,E1,ischemic,5,5.02,0.972,61.728395,3.29,3.765,0.1,0.16,0.372,0.37732,0.272


In [ ]:
data.shape

(1048574, 13)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048574 entries, 0 to 1048573
Data columns (total 13 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   Subject  1048574 non-null  object
 1   Type     1048574 non-null  object
 2   Cycle    1048574 non-null  object
 3   Time     1048574 non-null  object
 4   RR-I     1048574 non-null  object
 5   HR       1048574 non-null  object
 6   R-H      1048574 non-null  object
 7   P-H      1048574 non-null  object
 8   QRS      1048574 non-null  object
 9   PRQ      1048574 non-null  object
 10  QT       1048574 non-null  object
 11  QTC      1048574 non-null  object
 12  ST       1048574 non-null  object
dtypes: object(13)
memory usage: 104.0+ MB


In [30]:
features = ['Time', 'RR-I', 'HR', 'R-H', 'P-H', 'QRS', 'PRQ', 'QT', 'QTC', 'ST']

for i in features:
  data[i] = pd.to_numeric(data[i], errors='coerce')

In [ ]:
data.describe()

,Time,RR-I,HR,R-H,P-H,QRS,PRQ,QT,QTC,ST
count,1.048570e+06,1.048570e+06,1.048570e+06,1.048570e+06,975958.000000,1.043601e+06,924386.000000,1.013741e+06,1.013741e+06,991114.000000
mean,3.466671e+03,8.675903e-01,7.475288e+01,1.254072e+00,0.418706,9.976558e-02,0.167872,4.160308e-01,4.495847e-01,0.333431
std,2.107287e+03,2.283222e-01,2.533810e+01,4.193589e+00,3.835203,3.004013e-02,0.033897,7.149738e-02,6.148705e-02,0.070306
min,4.080000e-01,0.000000e+00,0.000000e+00,-1.250000e+01,-11.346440,2.800000e-02,0.000000,6.000000e-02,3.874000e-02,0.008000
25%,1.569905e+03,7.080000e-01,5.859375e+01,-1.100000e+00,-1.665000,8.000000e-02,0.144444,3.760000e-01,4.118252e-01,0.296000
50%,3.417056e+03,8.560000e-01,7.009346e+01,6.103500e-01,-0.075680,9.200000e-02,0.164000,4.080000e-01,4.376780e-01,0.332000
75%,5.304838e+03,1.024000e+00,8.474576e+01,1.928710e+00,0.605000,1.120000e-01,0.188000,4.440000e-01,4.703798e-01,0.364000
max,7.291732e+03,3.586800e+01,1.875000e+03,1.638000e+01,16.380000,1.636000e+00,0.460000,1.708000e+00,1.743220e+00,0.876000


In [ ]:
data.isnull().sum()

0
Subject         0
Type            0
Cycle           0
Time            4
RR-I            4
HR              4
R-H             4
P-H         72616
QRS          4973
PRQ        124188
QT          34833
QTC         34833
ST          57460
dtype: int64

In [37]:
data_2 = data.fillna({'RR-I':data['RR-I'].mean(),
                           'HR' :data['HR'].mean(),
                           'R-H':data['R-H'].mean(),
                           'P-H':data['P-H'].mean(),
                           'QRS':data['QRS'].mean(),
                           'PRQ':data['PRQ'].mean(),
                           'QT' :data['QT'].mean(),
                           'QTC':data['QTC'].mean(),
                           'ST':data['ST'].mean()
                           }
                     )

print("Total number of row: ",data_2.isnull().sum().sum())
print("Total number of columns: ",data_2.isna().any().sum())

Total number of row:  4
Total number of columns:  1


In [38]:
X= data_2.loc[:,data_2.columns != target]
y = data_2[target]

In [ ]:
y.value_counts()

ischemic      705983
healthy       284849
arrhythmic     57742
Name: Type, dtype: int64

In [39]:
 X_train, X_test, y_train, y_test = fun_split(X,y, test_size=0.2, feature= 'Subject')

  0%|          | 0/3 [00:00<?, ?it/s]

Train Percentage: 0.8011709235590431


In [ ]:
"""#testing if there is any mixing with training subject and testing subject
from tqdm.notebook import tqdm
train = list(X_train["Subject"])
test = list(X_test["Subject"])

dup = []

for i in tqdm(test):
  if i in train:
    dup.append(i)
print(dup)"""

'#testing if there is any mixing with training subject and testing subject\nfrom tqdm.notebook import tqdm\ntrain = list(X_train["Subject"])\ntest = list(X_test["Subject"])\n\ndup = []\n\nfor i in tqdm(test):\n  if i in train:\n    dup.append(i)\nprint(dup)'

In [40]:
X_train= X_train.loc[:,X_train.columns != 'Subject']
X_train= X_train.loc[:,X_train.columns != 'Time']
X_train= X_train.loc[:,X_train.columns != 'Cycle']

X_test= X_test.loc[:,X_test.columns != 'Subject']
X_test= X_test.loc[:,X_test.columns != 'Time']
X_test= X_test.loc[:,X_test.columns != 'Cycle']

In [ ]:
y_train.value_counts()

ischemic      563596
healthy       230515
arrhythmic     45976
Name: Type, dtype: int64

In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

smote = SMOTE()

X_train, y_train= smote.fit_resample(X_train, y_train)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
y_train.value_counts()

ischemic      464402
healthy       464402
arrhythmic    464402
Name: Type, dtype: int64

In [ ]:
X_train.head(-1)

,RR-I,HR,R-H,P-H,QRS,PRQ,QT,QTC,ST
0,0.932,64.377682,3.305000,3.810000,0.088000,0.160000,0.384000,0.397762,0.296000
1,1.004,59.760956,3.285000,3.810000,0.100000,0.160000,0.640000,0.638724,0.540000
2,0.996,60.240964,3.340000,3.830000,0.128000,0.148000,0.628000,0.629260,0.500000
3,1.016,59.055118,3.325000,3.810000,0.108000,0.172000,0.388000,0.384933,0.280000
4,0.972,61.728395,3.290000,3.765000,0.100000,0.160000,0.372000,0.377320,0.272000
...,...,...,...,...,...,...,...,...,...
1393200,1.044,57.471260,0.546880,0.016341,0.084000,0.134772,0.458772,0.449000,0.398772
1393201,1.140,52.631580,10.105774,8.086968,0.088451,0.199097,0.412161,0.386023,0.346677
1393202,0.900,66.666670,0.683092,-0.031241,0.095796,0.135796,0.407796,0.429855,0.347796
1393203,0.996,60.240960,0.765248,0.091418,0.096000,0.164431,0.444461,0.445350,0.376461


In [ ]:
X_test.head(-1)

,RR-I,HR,R-H,P-H,QRS,PRQ,QT,QTC,ST
0,0.856000,70.093458,2.275,-0.435,0.140000,0.132000,0.412000,0.445308,0.292000
1,0.796000,75.376884,1.905,-0.590,0.132000,0.128000,0.380000,0.425919,0.272000
2,0.916000,65.502183,1.860,-0.425,0.132000,0.128000,0.392000,0.409580,0.284000
3,1.388000,43.227666,0.955,-0.480,0.144000,0.132000,0.400000,0.339520,0.304000
4,0.680000,88.235294,2.200,-0.435,0.136000,0.128000,0.372000,0.451116,0.260000
...,...,...,...,...,...,...,...,...,...
177504,0.638889,93.913043,0.680,-0.315,0.063889,0.158333,0.402778,0.503910,0.350000
177505,0.652778,91.914894,0.660,-0.335,0.066667,0.163889,0.411111,0.508834,0.355556
177506,0.675000,88.888889,0.605,-0.345,0.066667,0.172222,0.413889,0.503770,0.358333
177507,0.647222,92.703863,0.795,-0.335,0.055556,0.161111,0.397222,0.493750,0.341667


In [36]:
X_train.isnull().sum()

0
RR-I        0
HR          0
R-H         0
P-H         0
QRS         0
PRQ         0
QT          0
QTC         0
ST      50206
dtype: int64

In [41]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier ()

rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[  3105    471   8190]
 [ 14062  10230  30042]
 [  3195     49 139142]]
              precision    recall  f1-score   support

  arrhythmic       0.15      0.26      0.19     11766
     healthy       0.95      0.19      0.31     54334
    ischemic       0.78      0.98      0.87    142386

    accuracy                           0.73    208486
   macro avg       0.63      0.48      0.46    208486
weighted avg       0.79      0.73      0.69    208486

Accurecy:  0.7313536640349951


In [43]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()

ada.fit(X_train, y_train)

y_pred = ada.predict(X_test)

In [44]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[  2824    963   7979]
 [  5506  27420  21408]
 [  2968    122 139296]]
              precision    recall  f1-score   support

  arrhythmic       0.25      0.24      0.24     11766
     healthy       0.96      0.50      0.66     54334
    ischemic       0.83      0.98      0.90    142386

    accuracy                           0.81    208486
   macro avg       0.68      0.57      0.60    208486
weighted avg       0.83      0.81      0.80    208486

Accurecy:  0.8131960899053173


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(X_train, y_train)

y_pred = GBC.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

In [ ]:

import xgboost as xgb
xgb_default = xgb.XGBClassifier()
xgb_default.fit(X_train,y_train)
y_pred = xgb_default.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[  6410    574    575]
 [  9252  16746  27137]
 [   325     18 116473]]
              precision    recall  f1-score   support

           0       0.40      0.85      0.54      7559
           1       0.97      0.32      0.48     53135
           2       0.81      1.00      0.89    116816

    accuracy                           0.79    177510
   macro avg       0.72      0.72      0.64    177510
weighted avg       0.84      0.79      0.75    177510

Accurecy:  0.7865979381443299
